In [10]:
import pandas as pd
import numpy as np

In [11]:
assay = pd.read_csv("../data/assay.csv")
collar = pd.read_csv("../data/collar.csv")
survey = pd.read_csv("../data/survey.csv")

In [12]:
from desurvey import mincurve_desurvey, simple_desurvey

In [18]:
def desurvey(survey, collar, interp_type="mincurve"):
    def get_coords(row):
        if np.isnan(row.length):
            icollar = collar[collar.DHID == row.DHID]
            return {"dhid":row.DHID, 
                "x":icollar.X.values[0], 
                "y":icollar.Y.values[0], 
                "z":icollar.Z.values[0]}
        else:
            if interp_type =="mincurve":
                x,y,z = mincurve_desurvey(row.length, row.AZIMUTH, row.DIP, row.AZIMUTH2, row.DIP2)
                return {"dhid":row.DHID,"x":x, "y":y, "z":-z}
            elif interp_type=="tangent":
                x,y,z = simple_desurvey(row.length, row.AZIMUTH, row.DIP)
                return {"dhid":row.DHID,"x":x, "y":y, "z":-z}
            else:
                raise ValueError(f"Invalid interpolation method: {interp_type}")
            

    bhid_groups = survey.groupby("DHID")
    survey["length"] = bhid_groups["DEPTH"].diff()
    survey["AZIMUTH2"] = bhid_groups["AZIMUTH"].shift(1)
    survey["DIP2"] = bhid_groups["DIP"].shift(1)
    coords = survey.apply(get_coords, axis=1, result_type="expand")
    coords["z"] = coords.groupby("dhid")["z"].cumsum()
    return coords

In [19]:
desurvey(survey, collar, interp_type="tangent")

,dhid,x,y,z
0,0.0,4.533054e+00,0.000000,99.727177
1,0.0,1.947928e-17,0.318121,94.082975
2,0.0,1.270782e-17,0.207534,91.264025
3,0.0,1.567230e-17,0.255948,88.449057
4,0.0,1.863215e-17,0.304286,85.638903
...,...,...,...,...
367,9.0,7.250227e-17,1.184052,77.485825
368,9.0,8.106984e-17,1.323971,69.154148
369,9.0,8.961440e-17,1.463514,60.845847
370,9.0,9.813395e-17,1.602649,52.563259
